# Data collection

This notebook will be used to collect data using the API.

API docs - https://www.api-football.com/documentation-v3

# Libraries

In [2]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
from pandasql import sqldf

# Functions

In [3]:
def api(param):
    
    # Define api url
    url = 'https://api-football-v1.p.rapidapi.com/v3/' + param
    
    # Authentication
    headers = {
        "X-RapidAPI-Key": key,
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers)
    
    return response.json()

# Initialization

In [4]:
# Read api key
with open('api-key', 'r') as f:
    key = f.read()
    
# List of countries
lstCount = ['France','Belgium','England','Germany','Italy','Netherlands','Portugal','Spain','Ukraine','Turkey']

# List of leagues
lstLeag = ['Premier League','La Liga','Serie A','Bundesliga','Ligue 1','Primeira Liga','Eredivisie','Süper Lig',\
           'Jupiler Pro League']

# Load data

## Seasons

In [5]:
# Season start and end
season_start_id = 2012
season_end_id = 2021

## Leagues

Run api

In [34]:
jLeagues = api('leagues')

Transform into dataframe

In [ ]:
# Normalize from json
dfLeagues = pd.json_normalize(jLeagues['response'])

# Rename columns
dfLeagues.columns = dfLeagues.columns.str.replace('.', '_')

# Filter leagues
dfLeagues = dfLeagues[dfLeagues.league_name.isin(lstLeag) & dfLeagues.country_name.isin(lstCount)]

# Save into file
dfLeagues.drop('seasons', axis=1).to_csv('leagues.csv', index=False)

# Get league id
# league_id = dfLeagues[(dfLeagues['country_name'] == 'England')\
#                       & (dfLeagues['league_name'] == 'Premier League')]['league_id'].values[0]

## Teams

In [ ]:
# Get list of leagues
dfLeagues = pd.read_csv('data/leagues.csv')
listLeagues = dfLeagues['league_id'].to_list()

# Initialise an empty data frame
dfTeams = pd.DataFrame()

# Clear log file
open('log/teams.log', 'w').close()

# Get teams for all seasons and all leagues
for season_id in range(season_start_id, 2022):
    for league_id in listLeagues:

        # Run api
        jTeams = api(f'teams?league={league_id}&season={season_id}')

        # Normalize from json
        dfTeamsTemp = pd.json_normalize(jTeams['response'])

        # Rename columns
        dfTeamsTemp.columns = dfTeamsTemp.columns.str.replace('.', '_')

        # Add leage and season
        dfTeamsTemp['league_id'] = league_id
        dfTeamsTemp['season_id'] = season_id
        
        # Merge temp data to the main df
        dfTeams = pd.concat([dfTeams,dfTeamsTemp])

        # Log
        with open('log/teams.log', 'a') as f:
            f.write(f'season_id = {season_id}, league_id = {league_id}\n')
        
# Save into file
dfTeams.to_csv('teams.csv', index=False)

## Players statistics

In [ ]:
# Season start and end
season_start_id = 2012
season_end_id = 2021

# Clear log file
open('log/players.log', 'w').close()

# Get list of leagues
dfLeagues = pd.read_csv('raw_data/leagues.csv')
listLeagues = dfLeagues['league_id'].to_list()

# Get teams for all seasons and all leagues
for season_id in range(season_start_id, 2022):
    
    # Initialise empty data frames
    dfPlayers = pd.DataFrame()
    dfPlayerStats = pd.DataFrame()
    
    for league_id in listLeagues:

        # Get total number of pages
        jPlayers = api(f'players?league={league_id}&season={season_id}')

        # Get total number of pages
        pages = jPlayers['paging']['total']

        # Get all pages
        for page_id in range(pages):

            # Get current page
            jPlayers = api(f'players?league={league_id}&season={season_id}&page={page_id+1}')

            # Get number of records
            records = len(jPlayers['response'])

            # Initialise empty data frames
            dfPlayersTemp = pd.DataFrame()
            dfPlayerStatsTemp = pd.DataFrame()

            # Get all records
            for record_id in range(records):

                # Get current page of players
                dfPlayersTemp = pd.json_normalize(jPlayers['response']).iloc[[record_id]].drop('statistics', axis=1)

                # Get current page of players' statistics
                dfPlayerStatsTemp = pd.json_normalize(pd.json_normalize(jPlayers['response'])\
                                                        .iloc[[record_id]]['statistics'].iloc[0])

                # Add player_id, league_id and season_id
                player_id = dfPlayersTemp['player.id'].iloc[0]
                dfPlayerStatsTemp['player_id'] = player_id
                dfPlayersTemp['league_id'] = league_id
                dfPlayersTemp['season_id'] = season_id
                dfPlayerStatsTemp['season_id'] = season_id

                # Merge the current page to the result dataframe
                dfPlayers = pd.concat([dfPlayers, dfPlayersTemp])
                dfPlayerStats = pd.concat([dfPlayerStats, dfPlayerStatsTemp])

                # Log
                with open('log/players.log', 'a') as f:
                    f.write(f'season_id = {season_id}, league_id = {league_id}, player_id = {player_id}, page_id = {page_id + 1}, pages = {pages}\n')

    # Rename columns
    dfPlayers.columns = dfPlayers.columns.str.replace('.', '_')
    dfPlayerStats.columns = dfPlayerStats.columns.str.replace('.', '_')

    # Save into file
    dfPlayers.to_csv('players_' + str(season_id) + '.csv',index=False)
    dfPlayerStats.to_csv('player_stats_' + str(season_id) + '.csv',index=False)

## Games

In [157]:
# jGames = api(f'fixtures/?league={league_id}&season={season_id}')

In [160]:
# # Normalize
# dfGames = pd.json_normalize(jGames['response'])

# # Rename columns
# dfGames.columns = dfGames.columns.str.replace('.', '_')

# # Add leage and season
# dfGames['league_id'] = league_id
# dfGames['season_id'] = season_id

# # Save into file
# dfGames.to_csv('games.csv',index=False)

/var/folders/hn/r6dxb61155x4z2mwg4dbmqtc0000gq/T/ipykernel_23302/2150249608.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfGames.columns = dfGames.columns.str.replace('.', '_')


## Game players statistics

In [161]:
# jGamePlayerStats = api(f'fixtures/players?fixture=710556')

In [ ]:
# pd.json_normalize(pd.json_normalize(jGamePlayerStats['response'])['players'][0])

## Game statistics

In [11]:
# jGameStat = api('fixtures?live=all')

In [ ]:
# pd.json_normalize(jGameStat['response'])

In [48]:
# # Initialise an empty data frame
# dfSquads = pd.DataFrame()

# # Get squads for all teams
# for team_id in dfTeams['team_id']:

#     # Run api
#     jSquads = api(f'players/squads?team={team_id}')

#     # Normalize
#     dfSquadsTemp = pd.json_normalize(pd.json_normalize(jSquads['response'])['players'][0])
    
#     # Add team_id column
#     dfSquadsTemp['team_id'] = team_id
    
#     # Append
#     dfSquads = pd.concat([dfSquads, dfSquadsTemp])

# # Save into file
# dfSquads.to_csv('squads.csv',index=False)

Appended team 33
Appended team 34
Appended team 38
Appended team 39
Appended team 40
Appended team 41
Appended team 42
Appended team 44
Appended team 45
Appended team 46
Appended team 47
Appended team 48
Appended team 49
Appended team 50
Appended team 51
Appended team 52
Appended team 55
Appended team 63
Appended team 66
Appended team 71


## Squads

## Transfers

In [27]:
# jTransfer = api('transfers?player=20355')

In [29]:
# pd.json_normalize(jTransfer['response'])

,update,transfers,player.id,player.name
0,2023-04-03T06:24:08+00:00,"[{'date': '2021-08-20', 'type': '€ 28M', 'team...",20355,A. Ramsdale


# Other

In [27]:
jPlayers = api(f'players?id=257&season=2012')

In [29]:
df1 = pd.json_normalize(pd.json_normalize(jPlayers['response']).iloc[[0]]['statistics'][0])

df1[df1['league.id'] == 135].iloc[:,10:]

,team.id,team.name,team.logo,league.id,league.name,league.country,league.logo,league.flag,league.season,games.appearences,...,fouls.drawn,fouls.committed,cards.yellow,cards.yellowred,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved
0,497,AS Roma,https://media-3.api-sports.io/football/teams/4...,135.0,Serie A,Italy,https://media-1.api-sports.io/football/leagues...,https://media-2.api-sports.io/flags/it.svg,2012,10,...,None,None,0,0,0,None,None,None,None,None
1,497,AS Roma,https://media-3.api-sports.io/football/teams/4...,135.0,Serie A,Italy,https://media-2.api-sports.io/football/leagues...,https://media-2.api-sports.io/flags/it.svg,2012,16,...,None,None,1,0,1,None,None,None,None,None


In [30]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 46 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   team.id                5 non-null      int64  
 1   team.name              5 non-null      object 
 2   team.logo              5 non-null      object 
 3   league.id              3 non-null      float64
 4   league.name            5 non-null      object 
 5   league.country         3 non-null      object 
 6   league.logo            3 non-null      object 
 7   league.flag            3 non-null      object 
 8   league.season          5 non-null      object 
 9   games.appearences      5 non-null      int64  
 10  games.lineups          5 non-null      int64  
 11  games.minutes          5 non-null      int64  
 12  games.number           0 non-null      object 
 13  games.position         5 non-null      object 
 14  games.rating           0 non-null      object 
 15  games.capt